<a href="https://colab.research.google.com/github/satish4github/dlutils/blob/master/tfrecords_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy
import requests
import os.path
from os import path
from shutil import copyfile
from google.colab import drive

In [0]:
class datasets():
    def load_cifar(self):
      return tf.keras.datasets.cifar10.load_data()

#TODO : yet to move following methods to utils
def is_file_exists(file_path):
      return path.exists(file_path)

def mkdirs(filepath):
      if not os.path.exists(filepath):
              os.makedirs(filepath)
def mount():
    drive.mount('/content/drive')

In [0]:
class TFRecord:
  base_path = '/content/drive/My Drive/models/tfrecords/'
  
  def __init__(self,base_path=base_path,is_mount_required=True):
    tf.enable_eager_execution()
    self.base_path = base_path
    self.is_mount_required = is_mount_required
    self.datasets = datasets()
    if is_mount_required :
      mount()

  def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

  def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

  def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
   
  def convert_to_tfrecords(self,images, labels,fileName):
      with tf.python_io.TFRecordWriter(fileName) as writer:
        if isinstance(images[0], numpy.ndarray):
          n = images.shape[0]
          reshape = images.reshape(n, -1)
          for i in range(n):
            feature = {'image': tf.train.Feature(float_list=tf.train.FloatList(value=reshape[i])),'label': tf.train.Feature(int64_list=tf.train.Int64List(value=labels[i]))}
            example=tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

  def parse_tfrecord(proto,h=32,w=32,c=3):
        dtype=tf.float32
        features = {'image': tf.FixedLenFeature([h * w * c], dtype),
                    'label': tf.FixedLenFeature([], tf.int64)}
        example = tf.io.parse_single_example(proto, features=features)
        x, y = example["image"], example['label']
        x = tf.reshape(x, [h, w, c])
        return x, y

  def load_cifar10(self):
      cifar10_tfrecord_path = self.base_path+'cifar10/'
      mkdirs(cifar10_tfrecord_path)
      train_tf_record_filepath = cifar10_tfrecord_path+'train.tfrecords'
      test_tf_record_filepath = cifar10_tfrecord_path+'test.tfrecords'

      if is_file_exists(train_tf_record_filepath) == False:      
          (x_train, y_train), (x_test, y_test) = self.datasets.load_cifar()
          self.convert_to_tfrecords(x_train,y_train,'train.tfrecords')
          self.convert_to_tfrecords(x_test,y_test,'test.tfrecords')
          copyfile('train.tfrecords', train_tf_record_filepath)
          os.remove('train.tfrecords')
          copyfile('test.tfrecords', test_tf_record_filepath)
          os.remove('test.tfrecords')
      return (train_tf_record_filepath,test_tf_record_filepath)


In [0]:
(train_tf_record_filepath,test_tf_record_filepath) = TFRecord().load_cifar10()
print(train_tf_record_filepath,'---',test_tf_record_filepath)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/models/tfrecords/cifar10/train.tfrecords --- /content/drive/My Drive/models/tfrecords/cifar10/test.tfrecords
